In [1]:
import pandas as pd
import random
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import nltk
from nltk.corpus import stopwords
from gensim.models.doc2vec import TaggedDocument
from sklearn.model_selection import train_test_split
from bs4 import BeautifulSoup
from gensim.models import Doc2Vec
from tqdm import tqdm
from sklearn import utils
from sklearn.metrics import accuracy_score, f1_score
tqdm.pandas(desc="progress-bar")
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
import sklearn, sklearn.ensemble, sklearn.tree, sklearn.semi_supervised, sklearn.discriminant_analysis
import sklearn.neural_network, sklearn.gaussian_process

import warnings
warnings.filterwarnings('ignore')

### Display Options for DFs

In [2]:
pd.set_option('display.max_colwidth', -1)

# Functions

In [3]:
# clean tweets from URLs and Mentions
def clean_text(text):
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'http.?://[^\s]+[\s]?', '', text)
    text = text.lower()
    return text

def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text):
        for word in nltk.word_tokenize(sent):
            if len(word) < 2:
                continue
            tokens.append(word.lower())
    return tokens

def add_sentiment(text, text2):
    if text2 == 4:
        return text + " positive"
    else:
        return text + " negative"
    
def feature_eng(tweet):
    step1 = re.sub(r'@([^:\s]+)', r'__MENTION__', tweet)
    output = re.sub(r'(https?://[^\s]+)', r'__URL__', step1)
    return output

def test_multiclass_classifiers(X_train, y_train, X_test, y_test):
    classifiers = [sklearn.naive_bayes.BernoulliNB, sklearn.tree.DecisionTreeClassifier,
        sklearn.tree.ExtraTreeClassifier, sklearn.ensemble.ExtraTreesClassifier,
        #sklearn.naive_bayes.GaussianNB, 
        sklearn.neighbors.KNeighborsClassifier,
        #klearn.semi_supervised.LabelPropagation, sklearn.semi_supervised.LabelSpreading,
        #sklearn.discriminant_analysis.LinearDiscriminantAnalysis, 
        sklearn.svm.LinearSVC,
        sklearn.linear_model.LogisticRegression, sklearn.linear_model.LogisticRegressionCV,
        sklearn.neural_network.MLPClassifier,  sklearn.neighbors.NearestCentroid,
        #sklearn.discriminant_analysis.QuadraticDiscriminantAnalysis, 
        #sklearn.neighbors.RadiusNeighborsClassifier, 
        sklearn.ensemble.RandomForestClassifier, sklearn.linear_model.RidgeClassifier, sklearn.linear_model.RidgeClassifierCV,
        #sklearn.svm.NuSVC,
        #sklearn.svm.SVC, sklearn.gaussian_process.GaussianProcessClassifier, 
        sklearn.ensemble.GradientBoostingClassifier,
        sklearn.gaussian_process.GaussianProcessClassifier, sklearn.svm.LinearSVC,
        sklearn.linear_model.LogisticRegression, sklearn.linear_model.LogisticRegressionCV,
        sklearn.linear_model.SGDClassifier, sklearn.linear_model.Perceptron,
        sklearn.linear_model.PassiveAggressiveClassifier]

    for clf in classifiers:
        print(clf)
        pipeline = Pipeline([('vect',CountVectorizer(max_df=0.5, ngram_range = (1,2), lowercase = True)),
                        (str(clf), clf())])
        pipeline.fit(X_train, y_train)

        y_predicted = pipeline.predict(X_test)

        score = sum(y_test == y_predicted)/len(y_test) * 100
        
        print("Score = {:.2f}".format(score))
        print()
        
def test_multilabel_classifiers(X_train, y_train, X_test, y_test):
    classifiers = [sklearn.tree.DecisionTreeClassifier,
    sklearn.tree.ExtraTreeClassifier,
    sklearn.ensemble.ExtraTreesClassifier,
    sklearn.neighbors.KNeighborsClassifier,
    sklearn.neural_network.MLPClassifier,
    sklearn.ensemble.RandomForestClassifier]
    
    for clf in classifiers:
        print(clf)
        pipeline = Pipeline([('vect',CountVectorizer(max_df=0.5, ngram_range = (1,2), lowercase = True)),
                        (str(clf), clf())])
        pipeline.fit(X_train, np.array(y_train))

        y_predicted = pipeline.predict(X_test)

        score = sum([np.prod(i) for i in np.array(y_predicted == y_test)])/len(y_test) * 100
        
        print("Score = {:.2f}".format(score))
        print()
    
    

# Research Data

In [4]:
import glob 

path = 'refilteredtweets/*.csv'
colnames = ['tweets', 'FP']

xlists = []
ylists = []

for fname in glob.glob(path):
    tweets = pd.read_csv(fname, skipinitialspace=True, usecols=colnames)
#     tweets['tweets'] = tweets['tweets'].apply(clean_text)
    xlists.append(tweets['tweets'].tolist())
    ylists.append(tweets['FP'].tolist())
    
    
X = [x for xlist in xlists for x in xlist] # the element, then the outer loop first, then inner loop
y = [y if y == 1.0 else 0.0 for ylist in ylists for y in ylist]

X = [feature_eng(x) for x in X]

print(len(X), len(y))

643 643


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [6]:
X_train[:10]

['Protect yourself from Zika! Wear protective clothing &amp; use approved repellents when outside. #Zika #ProtectYourself __URL__',
 'Got bit by a mosquito. Now all I can think of is #zika.  My brain.  sigh...',
 'Take some simple steps 2 protect yourself from the #Zika virus. __URL__ __MENTION__ __MENTION__ __URL__',
 'Take just 2 teaspoons of 100% natural RepelZika and your entire body is protected from mosquitoes.   #RepelZika… __URL__',
 'Fighting Zika in the US: The Battle Over GMO Mosquitoes - ABC News - __URL__ via __MENTION__',
 "Keep spraying all those toxic pesticides &amp; you'll kill more people than Zika will. __MENTION__ __MENTION__ __URL__",
 'Protect your family and pets from diseases like West Nile Virus, Zika Virus, and Malaria by getting your yard sprayâ\x80¦ __URL__',
 'Get the real scoop on #zika and what action you can take to protect yourself and others: __URL__',
 'See: Spike in sales of mosquito repellent products amid Zika concerns __URL__ __URL__',
 'Take a l

# Model 1 - Classify FP Tweets

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from time import time
from sklearn.pipeline import FeatureUnion

In [8]:
pipeline_1 = Pipeline([('vect',CountVectorizer(max_df=0.5, ngram_range = (1,2), lowercase = True)),
#                     ('tfidf',TfidfTransformer(norm='l1', use_idf=True)),
                    ('MNB',MultinomialNB(fit_prior=False))])
#                     ('LogReg', LogisticRegression(penalty='l1'))])

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score

scoring = ['precision_macro', 'recall_macro']

scores = cross_validate(pipeline_1, X, y, scoring=scoring, cv=5, return_train_score=True)

for k, v in scores.items():
    print(k,":", v)
    print()

fit_time : [0.03305006 0.02505922 0.02404213 0.02401996 0.02434874]

score_time : [0.01195478 0.00969028 0.00924897 0.00936747 0.00903106]

test_precision_macro : [0.74634503 0.77284946 0.79069767 0.79282151 0.79175258]

train_precision_macro : [0.98846913 0.99402985 0.9942407  0.99255952 0.99552239]

test_recall_macro : [0.76479832 0.73926139 0.78156103 0.77671556 0.7300813 ]

train_recall_macro : [0.98604989 0.98907104 0.99302495 0.9863388  0.99184783]



In [10]:
parameters = {
    'vect__lowercase': ('True', 'False'),   
}
grid_search = GridSearchCV(pipeline_1, parameters, cv=5, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("parameters:")
print(parameters)
t0 = time()
grid_search.fit(X_train, y_train)
print("done in %0.3fs" % (time() - t0))
print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

Performing grid search...
parameters:
{'vect__lowercase': ('True', 'False')}
Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


done in 1.110s
Best score: 0.813
Best parameters set:
	vect__lowercase: 'True'


[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    1.0s remaining:    0.7s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    1.1s finished


In [11]:
pipeline_1.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.5,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('MNB',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False))],
         verbose=False)

In [12]:
y_predicted = pipeline_1.predict(X_test)
print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

         0.0       0.94      0.82      0.87        88
         1.0       0.69      0.88      0.77        41

    accuracy                           0.84       129
   macro avg       0.81      0.85      0.82       129
weighted avg       0.86      0.84      0.84       129



# Run Model 1

In [13]:
import glob

In [14]:
for file in glob.glob("all_tweets/*"):
    tweets = list(pd.read_csv(file, lineterminator='\n')['tweet'])
    labels = pipeline_1.predict(tweets)
    
    
    df = pd.DataFrame({"tweet": tweets, "label": labels})
    filename = file.split("/")[1].split(".")[0] + "_labeled.csv"
    df.to_csv("classified_tweets/{}".format(filename), index=False)

In [15]:
for file in glob.glob("classified_tweets/*"):
    print(file)
    
    df = pd.read_csv(file, lineterminator='\n')
    num_tweets = len(df)
    num_fp_tweets = sum(df['label'] == 1)
    
    print("Num tweets = {}".format(num_tweets))
    print("Num fp tweets = {}".format(num_fp_tweets))
    print("Percentage = {:.1f}".format(num_fp_tweets/num_tweets*100))
    print()

classified_tweets/tweets_pronouns_labeled.csv
Num tweets = 33533
Num fp tweets = 20625
Percentage = 61.5

classified_tweets/tweets_VBG_labeled.csv
Num tweets = 11415
Num fp tweets = 2084
Percentage = 18.3

classified_tweets/tweets_VB_labeled.csv
Num tweets = 4122
Num fp tweets = 616
Percentage = 14.9

classified_tweets/tweets_VBD_labeled.csv
Num tweets = 4299
Num fp tweets = 760
Percentage = 17.7

classified_tweets/tweets_VBN_labeled.csv
Num tweets = 6589
Num fp tweets = 1069
Percentage = 16.2

classified_tweets/tweets_VBZ_labeled.csv
Num tweets = 5185
Num fp tweets = 1049
Percentage = 20.2

classified_tweets/tweets_VBP_labeled.csv
Num tweets = 4781
Num fp tweets = 866
Percentage = 18.1



# Model 2 - Multiple Classification
Using only FP tweets from labeled data as training data

## Try 1

In [16]:
import numpy as np

In [17]:
y_cols = ["Observe mosquito", "Mosquito Bite", "Observe spraying", "General spraying observation",
        "Spraying for self", "Spraying for someone else", "Other kind of intervention",
        "Environmental consequences", "Conspiracies"]

In [18]:
df = pd.DataFrame()
for file in glob.glob("refilteredtweets/*"):
    df = pd.concat([df, pd.read_csv(file)], ignore_index=True)

In [19]:
fp_tweets = df[df["FP"] == 1].fillna(0)
X = fp_tweets["tweets"]
y = fp_tweets[y_cols]

#Fix value
y.loc[y["Spraying for someone else"] == "1?", "Spraying for someone else"] = 1.0
y = y.astype("float64")

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

### Multi class approach
* 9 classes: 0 to 8

In [20]:
y_train = np.argmax(np.array(y_train), axis=1)
y_test = np.argmax(np.array(y_test), axis=1)

In [21]:
values, counts = np.unique(y_train, return_counts=True)
for i, j in zip(values, counts):
    print("{}: {}".format(i, j))

0: 114
1: 9
2: 7
3: 7
4: 21
6: 2
7: 17
8: 6


In [22]:
pipeline = Pipeline([('vect',CountVectorizer(max_df=0.5, ngram_range = (1,2), lowercase = True)),
                    ('MNB',MultinomialNB(fit_prior=False))])

In [23]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.5,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('MNB',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False))],
         verbose=False)

In [24]:
y_predicted = pipeline.predict(X_test)
print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

           0       0.97      0.71      0.82        41
           1       0.00      0.00      0.00         1
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           7       0.17      0.33      0.22         3
           8       0.00      0.00      0.00         0

    accuracy                           0.65        46
   macro avg       0.16      0.15      0.15        46
weighted avg       0.87      0.65      0.74        46



In [25]:
# test_multiclass_classifiers(X_train, y_train, X_test, y_test)

### Multi label approach
* 9 labels for each row: in a form of a list [0/1, 0/1, 0/1, 0/1, 0/1, 0/1, 0/1, 0/1, 0/1]

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=42)

In [27]:
for i in range(len(y_train)):
    if(sum(y_train.iloc[i]) >= 2):
        print(list(y_train.iloc[i]))

[0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0]
[1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]
[1.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0]


In [28]:
test_multilabel_classifiers(X_train, y_train, X_test, y_test)

<class 'sklearn.tree.tree.DecisionTreeClassifier'>
Score = 56.52

<class 'sklearn.tree.tree.ExtraTreeClassifier'>
Score = 63.04

<class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
Score = 65.22

<class 'sklearn.neighbors.classification.KNeighborsClassifier'>
Score = 63.04

<class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>
Score = 63.04

<class 'sklearn.ensemble.forest.RandomForestClassifier'>
Score = 63.04



## Try 2

### Add one more class: Not classified

In [29]:
y.sum()

Observe mosquito                8.0 
Mosquito Bite                   15.0
Observe spraying                7.0 
General spraying observation    8.0 
Spraying for self               28.0
Spraying for someone else       1.0 
Other kind of intervention      3.0 
Environmental consequences      23.0
Conspiracies                    9.0 
dtype: float64

In [30]:
y_cols.append("Not classified")
new_y = pd.DataFrame(columns=y_cols)

In [31]:
for i in range(len(y)):
    row = list(y.iloc[i])
    if row == [0]*9:
        row.append(1.0)
    else:
        row.append(0.0)
    new_y.loc[i] = row

In [32]:
y = new_y

In [33]:
y.sum()

Observe mosquito                8.0  
Mosquito Bite                   15.0 
Observe spraying                7.0  
General spraying observation    8.0  
Spraying for self               28.0 
Spraying for someone else       1.0  
Other kind of intervention      3.0  
Environmental consequences      23.0 
Conspiracies                    9.0  
Not classified                  136.0
dtype: float64

### Multi classification - 10 labels: 9 for each class, and 1 additional for unclassified rows

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [35]:
y_train = np.argmax(np.array(y_train), axis=1)
y_test = np.argmax(np.array(y_test), axis=1)

In [36]:
values, counts = np.unique(y_train, return_counts=True)
for i, j in zip(values, counts):
    print("{}: {}".format(i, j))

0: 7
1: 8
2: 6
3: 6
4: 16
6: 2
7: 17
8: 5
9: 93


In [37]:
pipeline = Pipeline([('vect',CountVectorizer(max_df=0.5, ngram_range = (1,2), lowercase = True)),
                    ('MNB',MultinomialNB(fit_prior=False))])

In [38]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=0.5,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 2), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('MNB',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=False))],
         verbose=False)

In [39]:
y_predicted = pipeline.predict(X_test)
print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         0
           1       0.25      0.50      0.33         2
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           7       0.17      0.33      0.22         3
           8       0.00      0.00      0.00         0
           9       0.98      0.67      0.79        63

    accuracy                           0.64        69
   macro avg       0.15      0.17      0.15        69
weighted avg       0.91      0.64      0.74        69



### Binary classification: multi label

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state=42)

In [41]:
test_multilabel_classifiers(X_train, y_train, X_test, y_test)

<class 'sklearn.tree.tree.DecisionTreeClassifier'>
Score = 46.38

<class 'sklearn.tree.tree.ExtraTreeClassifier'>
Score = 50.72

<class 'sklearn.ensemble.forest.ExtraTreesClassifier'>
Score = 57.97

<class 'sklearn.neighbors.classification.KNeighborsClassifier'>
Score = 59.42

<class 'sklearn.neural_network.multilayer_perceptron.MLPClassifier'>
Score = 47.83

<class 'sklearn.ensemble.forest.RandomForestClassifier'>
Score = 60.87



### Binary classification: one label vs. rest

In [42]:
for i in y_cols:
    print(i, "vs. Rest")
    temp_y = y[i]
    X_train, X_test, y_train, y_test = train_test_split(X, temp_y, test_size = 0.3, random_state=42)
    
    pipeline.fit(X_train, y_train)
    y_predicted = pipeline.predict(X_test)
    print(classification_report(y_predicted, y_test))
    print()

Observe mosquito vs. Rest
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99        69
         1.0       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        69
weighted avg       1.00      0.99      0.99        69


Mosquito Bite vs. Rest
              precision    recall  f1-score   support

         0.0       0.98      0.95      0.97        66
         1.0       0.40      0.67      0.50         3

    accuracy                           0.94        69
   macro avg       0.69      0.81      0.73        69
weighted avg       0.96      0.94      0.95        69


Observe spraying vs. Rest
              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99        69
         1.0       0.00      0.00      0.00         0

    accuracy                           0.99        69
   macro avg       0.50      0.49      0.50        

### Multi classification: combine 0-1, 2-3, 4-5, 6-8, 9 (reduce from 9 to 5 labels)

New labels:
* 0: "Observe mosquito" and "Mosquito Bite"
* 1: "Observe spraying" and "General spraying observation"
* 2: "Spraying for self" and "Spraying for someone else"
* 3: "Other kind of intervention", "Environmental consequences", and "Conspiracies"
* 4: Not classified

In [43]:
new_y = pd.DataFrame(columns=[0, 1, 2, 3, 4])

In [44]:
col_0 = y["Observe mosquito"] + y["Mosquito Bite"]
col_0[col_0 == 2] = 1
new_y[0] = col_0

col_1 = y["Observe spraying"] + y["General spraying observation"]
col_1[col_1 == 2] = 1
new_y[1] = col_1

col_2 = y["Spraying for self"] + y["Spraying for someone else"]
col_2[col_2 == 2] = 1
new_y[2] = col_2

col_3 = y["Other kind of intervention"] + y["Environmental consequences"] + y["Conspiracies"]
col_3[col_3 == 2] = 1
new_y[3] = col_3

new_y[4] = y["Not classified"]

In [45]:
new_y.head()

,0,1,2,3,4
0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,1.0
2,1.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,1.0


In [46]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size = 0.3, random_state=42)

In [47]:
y_train = np.argmax(np.array(y_train), axis=1)
y_test = np.argmax(np.array(y_test), axis=1)

In [48]:
values, counts = np.unique(y_train, return_counts=True)
for i, j in zip(values, counts):
    print("{}: {}".format(i, j))

0: 15
1: 12
2: 16
3: 24
4: 93


In [49]:
pipeline_2 = Pipeline([('vect',CountVectorizer(max_df=0.5, ngram_range = (1,2), lowercase = True)),
                    ('MNB',MultinomialNB(fit_prior=False))])
pipeline_2.fit(X_train, y_train)
y_predicted = pipeline_2.predict(X_test)
print(classification_report(y_predicted, y_test))

              precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         1
           3       0.44      0.80      0.57         5
           4       0.95      0.69      0.80        59

    accuracy                           0.70        69
   macro avg       0.40      0.45      0.41        69
weighted avg       0.88      0.70      0.77        69



## Run both classifier on all tweets

In [79]:
for file in glob.glob("all_tweets/*"):
    print(file)
    
    tweets = pd.read_csv(file, lineterminator='\n')["tweet"]
    tweets_fp = tweets[pipeline_1.predict(tweets) == 1]
    tweets_final = tweets_fp[pipeline_2.predict(tweets_fp) != 4]
    
    new_file = "final_tweets/{}".format(file.split("/")[1])
    pd.DataFrame(columns=["tweets"], data=list(tweets_final)).to_csv(new_file, index=False)
    
    print("All tweets count: {}".format(len(tweets)))
    print("Final tweets count: {}".format(len(tweets_final)))
    print("Percentage: {:.1f}%".format(len(tweets_final)/len(tweets) * 100))
    print()

all_tweets/tweets_VBD.csv
All tweets count: 4299
Final tweets count: 167
Percentage: 3.9%

all_tweets/tweets_VBG.csv
All tweets count: 11415
Final tweets count: 616
Percentage: 5.4%

all_tweets/tweets_VBZ.csv
All tweets count: 5185
Final tweets count: 138
Percentage: 2.7%

all_tweets/tweets_VB.csv
All tweets count: 4122
Final tweets count: 93
Percentage: 2.3%

all_tweets/tweets_VBN.csv
All tweets count: 6589
Final tweets count: 200
Percentage: 3.0%

all_tweets/tweets_VBP.csv
All tweets count: 4781
Final tweets count: 146
Percentage: 3.1%

all_tweets/tweets_pronouns.csv
All tweets count: 33533
Final tweets count: 5649
Percentage: 16.8%

